# Getting Started with PubChemPy
This notebooks is designed to introduced you to PubChemPy, a library for working with [PubChem](https://www.example.com) resource. To use pubchempy, you'll need to either use the command

```pip install pubchempy```

on your command line or use the command

```!pip install pubchempy```

in the first coding cell in this notebook.

In [ ]:
!pip install pubchempy

It's not enough to have it installed on your computer. You need to tell the Jupyter notebook to access the library.

In [ ]:
import pubchempy as pcp

We are just going to look at a few things that you can do with PubChemPy, which accesses the [PubChem database](https://pubchem.ncbi.nlm.nih.gov/). We'll learn
1. How to access a molecule using its chemical ID#.
2. How to access a molecule by name
3. Some of the things we can learn about the molecule once we have its chemical ID#
4. How to visualize the molecule

We'll start looking at a molecule called NAD+ that I worked with almost every day in graduate school. It looks like this and its compound ID# is 5892.

![2D image of NAD+](images/NAD.png "The 2D structure of redox cofactor NAD+")

In [ ]:
pcp.get_compounds('aspirin', 'name', record_type='3d')

In [ ]:
molecule = pcp.Compound.from_cid(2244)

In [ ]:
print(molecule.molecular_weight)

In [ ]:
print(molecule.iupac_name)
print(molecule.molecular_formula)
# print(molecule.synonyms)

In [ ]:
# Visualize the aspirin in 3D

import py3Dmol
py3Dmol.view()
view = py3Dmol.view(width = 680, height = 250, query ='cid:2244', viewergrid = (1,3), linked = True)

view.setStyle({'line': {'linewidth': 8}}, viewer = (0,0))
view.setStyle({'stick': {'colorscheme':'cyanCarbon'}}, viewer = (0,1))
view.setStyle({'sphere': {}}, viewer = (0,2))

view.setBackgroundColor('#ebf4fb', viewer = (0,0))
view.setBackgroundColor('#cda9fc', viewer = (0,1))
view.setBackgroundColor('#e6e6e6', viewer = (0,2))

## Lipinski's Rule of 5

We can use PCP to get the values for Lipinski's rule of 5 for a compound in the PubChem database directly.

In [ ]:
# How could we make this more versatile?

drugname = 'aspirin'  # then replace 'aspirin' with drugname in each of the identifier lines

HBA = pcp.get_properties(
  properties = 'HBondAcceptorCount',
  identifier = "aspirin",
  namespace = "name"
  )
HBD = pcp.get_properties(
  properties = 'HBondDonorCount',
  identifier = "aspirin",
  namespace = "name"
  )
MW = pcp.get_properties(
  properties = 'MolecularWeight',
  identifier = "aspirin",
  namespace = "name"
  )
XLP = pcp.get_properties(
  properties = 'XlogP',
  identifier = "aspirin",
  namespace = "name"
  )
print(HBA, '\n', HBD, '\n', MW, '\n', XLP)

In [ ]:
# Create a list variable to hold all of the properties you want to explore
properties = ['HBondAcceptorCount', 'HBondDonorCount', 'MolecularWeight', 'XlogP']
properties2 = ['MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IsomericSMILES', 'InChI', 'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 'EffectiveRotorCount3D', 'ConformerCount3D']

In [ ]:
import pandas as pd
Lip5 = pcp.get_properties(properties, 'aspirin', 'name', as_dataframe = True)
Lip5

In [ ]:
AllProps = pcp.get_properties(properties2, 'aspirin', 'name', as_dataframe = True)
AllProps

## Collecting data for >1 drug

You can do everything above just by looking at the [PubChem page for aspirin](https://pubchem.ncbi.nlm.nih.gov/#query=aspirin). We're going to step things up one level at a time. We are going to collect data for four more drugs, then collect them all into a single data structure, as dataframe.

In [ ]:
# Repeat this process for three more drugs
pen = pcp.get_properties(properties, 'penicillin', 'name', as_dataframe = True)
vioxx = pcp.get_properties(properties, 'vioxx', 'name', as_dataframe = True)
strep = pcp.get_properties(properties, 'streptomycin', 'name', as_dataframe = True)
doxy = pcp.get_properties(properties, 'doxycycline', 'name', as_dataframe = True)

In [ ]:
result_df = pd.concat([Lip5, pen, vioxx, strep, doxy], ignore_index=True)
result_df

In [ ]:
# make a list of drug names to add into a new dataframe column
drug_name = ['aspirin', 'penicillin', 'vioxx', 'streptomycin', 'doxycycline']

# add a new column to the dataframe with the drug names
result_df['name'] = drug_name
result_df

In [ ]:
# reorder the columns
result_df = result_df[['name', 'MolecularWeight', 'XLogP', 'HBondDonorCount', 'HBondAcceptorCount']]
result_df

## Automating the process with a loop

One of my research students saw this at our Monday research meeting and suggested a loop to extend the process. You could easily to this for 100 or 1000 drugs.

In [ ]:
import pandas as pd
import pubchempy as pcp

properties = ['HBondAcceptorCount', 'HBondDonorCount', 'MolecularWeight', 'XlogP']
druglist = ['aspirin', 'penicillin', 'vioxx', 'streptomycin', 'doxycycline']

data = {}
results = pcp.get_properties(properties, druglist[0], 'name')
data['Drug Name'] = druglist[0]
data.update(results[0])
df = pd.DataFrame.from_dict([data]) # initializes the dataframe that can then be added to
df

for drug in druglist[1:len(druglist)]: # starts at 1 to avoid repeating the first drug
    datatemp = {} # creates a temporary dictionary for this one drug in the loop
    datatemp['Drug Name'] = drug # adds drug name to the dictionary of results so there is a column for drug name
    results = pcp.get_properties(properties, drug, 'name')
    datatemp.update(results[0]) # concatenates dictionaries
    df2 = pd.DataFrame.from_dict([datatemp]) # creates temporary dataframe
    df = pd.concat([df, df2], ignore_index = True)

df


## Downloading chemical structures with pubchempy


In [ ]:
pcp.download('PNG', 'aspirin.png', 'aspirin', 'name', overwrite=True, image_size='large')

## If we have time

In exploring pubchempy, I realized that it does not make sense to create a dataframe with one line in it. Dataframes are created to hold lots of information. So we will explore this briefly.

In [ ]:
# We will expand the list of properties to include molecular formula and molecular weight.

properties = ['MolecularFormula', 'MolecularWeight', 'HBondAcceptorCount', 'HBondDonorCount', 'MolecularWeight', 'XlogP']

# Next we will do a property search for anything that is structurally related to the molecule codeine.

codeine_smiles = 'CN1CC[C@]23[C@@H]4[C@H]1CC5=C2C(=C(C=C5)OC)O[C@H]3[C@H](C=C4)O'
p = pcp.get_properties(properties, codeine_smiles, 'smiles', searchtype='superstructure', as_dataframe = True)
p

In [ ]:
results = pcp.get_compounds('C6H12O6', 'formula') # this time use the molecular formula for sugar and then use formula as the identifier type

In [ ]:
sugars_df = pcp.get_properties(properties, 'C6H12O6', 'formula', as_dataframe = True)

In [ ]:
sugars_df